# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [ ]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget
!pip install nltk

In [ ]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government (2).zip'

In [ ]:
#!unzip government.zip

In [ ]:
# imports
# Put all your imports here


In [ ]:
from whoosh import index, writing, qparser, scoring
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import *
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import nltk
words =nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
DATA_DIR = "government"

DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")

# Put other path constants here


## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]
MAP- Mean Average Precision


### Q1 (b): Provide answer to Q1 (b) here [markdown cell]
This is the most efficient way to evaluate, by looking at the precision of the results from queries

## Question 2

### Q2 (a): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q2 = createIndex(mySchema)

def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()



In [ ]:
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]
addFilesToIndex(INDEX_Q2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
#INDEX_Q2 = None # Replace None with your index for Q2
#QP_Q2 = None # Replace None with your query parser for Q2
#SEARCHER_Q2 = None # Replace None with your searcher for Q2
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [ ]:
# define a query parser for the field "file_content" in the index
QP_Q2 = QueryParser("file_content", schema=INDEX_Q2.schema)
SEARCHER_Q2 = INDEX_Q2.searcher()
sampleQuery = QP_Q2.parse("mining gold silver coal")
sampleQueryResults = SEARCHER_Q2.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)


G00-90-0342721 0 26.64539777117502


In [ ]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2)

map                      1       0.0000
map                      2       0.0000
map                      4       0.0312
map                      6       0.0000
map                      7       0.0000
map                      9       0.0000
map                      10      0.1667
map                      14      0.2500
map                      16      0.0000
map                      18      1.0000
map                      22      0.2000
map                      24      1.0000
map                      26      0.1111
map                      28      0.0000
num_q                    all     14.0000
num_ret                  all     151.0000
num_rel                  all     33.0000
num_rel_ret              all     7.0000
map                      all     0.1971
gm_map                   all     0.0015
Rprec                    all     0.1667
bpref                    all     0.1508
recip_rank               all     0.2196
iprec_at_recall_0.00     all     0.2196
iprec_at_recall_0.10     all     0.2

In [ ]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1": #only display if the relationship exists ie.1
                print(i.rstrip())

printRelName(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4, "10")

---------------------------Topic_id and Topic_phrase----------------------------------
10 Physical Fitness
---------------------------Return documents----------------------------------
10 Q0 G00-35-1295262 0 18.042416 test
10 Q0 G00-27-1222379 1 17.270961 test
10 Q0 G00-04-0412407 2 16.728955 test
10 Q0 G00-57-3463009 3 16.658719 test
10 Q0 G00-15-2555140 4 16.005752 test
10 Q0 G00-84-0557790 5 15.506725 test
10 Q0 G00-05-0768168 6 15.217199 test
10 Q0 G00-71-1781567 7 15.069475 test
10 Q0 G00-89-1575855 8 14.177060 test
10 Q0 G00-05-3904373 9 14.168232 test
10 Q0 G00-95-0287843 10 13.253680 test
10 Q0 G00-04-0014570 11 12.663917 test
10 Q0 G00-59-3459439 12 11.927107 test
10 Q0 G00-12-1790785 13 11.170895 test
10 Q0 G00-22-0689568 14 11.165705 test
10 Q0 G00-53-0998741 15 10.311568 test
10 Q0 G00-75-2904152 16 9.771173 test
10 Q0 G00-47-2191998 17 8.877228 test
10 Q0 G00-92-2864043 18 8.693394 test
10 Q0 G00-63-2233776 19 8.547600 test
10 Q0 G00-27-1315760 20 8.535124 test
10 Q0 G00-1

### Q2 (b): Provide answer to Q2 (b) here [markdown cell
map( Mean Average Precision) for all the input queries is  0.1971

### Q2 (c): Provide answer to Q2(c) here [markdown cell]
Some did really bad:
Query 1,2,6,7,9,16,28 have 0 accuracy, no returned documents are the actual relevant documents, Query 19 has two relevant documents but there was no document returned

Some did good:
Query 18,24 have 100% of accuracy, all return with relevant document.


## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

I pick query 6 "physical therapists". This query's output has 0 accuracy.   

How can I obtain information about training, licensing, and
skills needed for physical therapists?

False negative: G00-10-0106475, this is an actual document displaying the Physical Therapy Board. This document doesn't include any complete phrase of "physical thrapists" (mentioned physical many times, 0 mention of thrapists), but mentioned "physical therapy" multiple times. The phrase "physical therapy" is mentioned in title(first line) of the document.  The current module failed to consider cognates of words, the location of the phrase appearance. Hence the model should be modified to understand associated words and related terms for more accurate results, and count in the number of unrelated words surrounded so that unrelated documents can be filtered out.

False positive: G00-26-3134051, this is a Indian Health Service job posting site that included a brief description of its staff memebers that included the phrase "physical therapists" once. This document has the highest Whoosh score because it was the only document that found the complete phrase as requested. Even though the phrase appearched only once and at the middle of a paragraph, surrounded by other irrelevant words, the module still ranked this with the highest score.  


### Q3 (b): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = StemmingAnalyzer()))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q3 = createIndex(mySchema)

def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]
addFilesToIndex(INDEX_Q3, filesToIndex)

# define a query parser for the field "file_content" in the index
QP_Q3 = QueryParser("file_content", schema=INDEX_Q3.schema)

SEARCHER_Q3 = INDEX_Q3.searcher()

def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1": #only display if the relationship exists ie.1
                print(i.rstrip())

printRelName(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3, "6")



already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.
---------------------------Topic_id and Topic_phrase----------------------------------
6 physical therapists
---------------------------Return documents----------------------------------
6 Q0 G00-17-0942793 0 19.190277 test
6 Q0 G00-71-1781567 1 18.762616 test
6 Q0 G00-62-0503229 2 17.574948 test
6 Q0 G00-32-3515764 3 16.668511 test
6 Q0 G00-66-3921879 4 16.501505 test
6 Q0 G00-29-3538849 5 16.056030 test
6 Q0 G00-77-4032514 6 15.875191 test
6 Q0 G00-59-0786269 7 15.246322 test
6 Q0 G00-60-3914816 8 12.427533 test
6 Q0 G00-26-3134051 9 12.200945 test
6 Q0 G00-79-0620805 10 12.139592 test
6 Q0 G00-21-0649032 11 7.771533 test
6 Q0 G00-45-4032177 12 7.396421 test
---------------------------Relevant documents----------------------------------
6 0 G00-10-0106475 1


In [ ]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3)

map                      1       0.0000
map                      2       0.5000
map                      4       0.5375
map                      6       0.0000
map                      7       0.0000
map                      9       0.0714
map                      10      0.2500
map                      14      1.0000
map                      16      0.0000
map                      18      1.0000
map                      19      0.5000
map                      22      0.0370
map                      24      1.0000
map                      26      0.0771
map                      28      0.0711
num_q                    all     15.0000
num_ret                  all     453.0000
num_rel                  all     35.0000
num_rel_ret              all     15.0000
map                      all     0.3363
gm_map                   all     0.0176
Rprec                    all     0.3000
bpref                    all     0.3000
recip_rank               all     0.4389
iprec_at_recall_0.00     all     0.

In [ ]:
# INDEX_Q3 = None # Replace None with your index for Q3 ok
# QP_Q3 = None # Replace None with your query parser for Q3
# SEARCHER_Q3 = None # Replace None with your searcher for Q3

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]
I modified the analyzer from RegexAnalyzer to StemmingAnalyzer. StemmingAnalyzer incorporates lower case filter, an optional stop filter, and a stemming filter on top of the regular expression Tokenizer. This creates more flexibility in the search range. Overall, the MAP value increased even though there were a couple queries that got worse. Query 6 remained the same MAP of 0.

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]
Yes

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]
Yes

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]
It was a good improvement that matched with my . Most of queries have a high MAP value. Even though query 6 still has MAP=0, the top scoring document is more related to the actual desired doc( same website but different page)

## Question 4

In [ ]:
# import nltk
# from nltk import wordnet
# # 1. tried incorporating wordnet to find synonumns of query words. ie to enlarge the topicqueries in the pyTrecEval, however, ended with lower map
# def geysyn(topicquery):
#   return [st.lemma_names()[0] for st in wordnet.synsets(topicquery)]

# #2. tried scoring the same query with two different algotherms, tried combinations from three whoosh scoring algotherms (BM25F, frequency, TD_IDF) & combine 
#   #scoring values together. I created two searchers for implementations of two algotherms and added additional searcher2 field in pyTrec using the method below in ptTrecEval
# Results.upgrade_and_extend(results2) # results2 is from a different searcher(searcher2) 

# #3. tried factory along with orgroup, ended with lower MAP
# og = qparser.OrGroup.factory(#anconstant)

# #4 tried different sequence of the analyzer, this is the best version
#  file_content = TEXT(analyzer = RegexTokenizer()| LowercaseFilter() | IntraWordFilter()| StopFilter()
#                    |CustomFilter(LancasterStemmer().stem )))

# #5 tried adjusting BM25F scoring constants.
# #6 tried incorporating list of common words from NLTK library to update the stopfilter but ended with lower MAP
# #7 tried stemfilter from Whoosh, lemmatizer, ended with lower MAP








In [ ]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse( topic_phrase)
            topicResults = searcher.search( topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [ ]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
import nltk
from nltk.stem import *
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  #stemmer = StemFilter(Cistem())
                  
                  file_content = TEXT(analyzer = RegexTokenizer()| LowercaseFilter() | IntraWordFilter()| StopFilter()
                   |CustomFilter(LancasterStemmer().stem )))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q4 = createIndex(mySchema)

def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]
addFilesToIndex(INDEX_Q4, filesToIndex)




already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
# define a query parser for the field "file_content" in the index
og = qparser.OrGroup
QP_Q4 = qparser.QueryParser("file_content", schema=INDEX_Q4.schema,group = og)
QP_Q4.add_plugin(qparser.FuzzyTermPlugin())

SEARCHER_Q4 = INDEX_Q4.searcher(weighting = scoring.BM25F(B=0.5, content_B=1.0, K1=1.5)) 


def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
      
          
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1": #only display if the relationship exists ie.1
                print(i.rstrip())

#printRelName(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4, "6")

pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4)

num_q                    1       1.0000
num_ret                  1       481.0000
num_rel                  1       5.0000
num_rel_ret              1       5.0000
map                      1       0.0610
gm_map                   1       -2.7961
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0526
iprec_at_recall_0.00     1       0.1034
iprec_at_recall_0.10     1       0.1034
iprec_at_recall_0.20     1       0.1034
iprec_at_recall_0.30     1       0.1034
iprec_at_recall_0.40     1       0.1034
iprec_at_recall_0.50     1       0.1034
iprec_at_recall_0.60     1       0.1034
iprec_at_recall_0.70     1       0.0412
iprec_at_recall_0.80     1       0.0412
iprec_at_recall_0.90     1       0.0365
iprec_at_recall_1.00     1       0.0365
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.10

### Please answer the following questions here
(a) A clear list of all final modifications made. 
 1. Analyzer modified - implemented basic regex tokenizer then, in the sequence of LowercaseFilter, IntraWordFilter, StopFilter, Stem filter from NLTK (sequence matter)
 2. LowercaseFilter
 3. IntrawordFilter
 4. stopFilter
 5. Stemfilter from NLTK
 6. modifed queryparser to add ORgroup 
 7. adjust the BM25F constant term values b= 0.55, k=1.5

(b)  Why each modification was made – how did it help?  
1. the regular expression tokenizer was too basic as it doesn't consider many possibilies of words hence limited the search, more steps of text processing should be implemented.  I tested multiple sequences, this sequence produces the highest MAP 
2. lowercasefilter implemented to keep all tokens in lowercase, easiler for other filters to be applied
3. intrawordfilter created more possibilities of words by considering different transformations/combinations of subwords. Whoosh indicated Intrawordfilter should be placed before lowercase, but I got a higher MAP value for placing the intrawordfilter right behind lowercase, and got a slightly lower MAP value if I removed the intrawordfilter completely. Hence, after multiple rounds of testing, this filter is placed right after lowercasefilter.
4. stopfilter implemented to remove all common words to be put into the index. I tried to enlarger the common words lists, but ended with lower MAP hence just applied the most basic stopfilter
5. Tried stemfilter from Whoosh, lemmatizer and stemfilter from NLTK. The stemfilter from NLTK performed the best amongst them and I chose to apply that.
6. allowing queryparser to take OR statement when searching creating more possibilities in the output of related documents. As provened, more documents were found when OR statement is added.
7. Adjusted the BM25F constant term value. According to the BM25F formula, a(relatively) small B & large K could potentially improve the score. After trial and error, I modified the constants.

(c)  The  final  MAP  performance  that  these  modifications  attained.
0.3982

## Validation

In [ ]:
# Run the following cells to make sure your code returns the correct value types

In [ ]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [ ]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [ ]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation

In [ ]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
